# RecoMIDI (ver. 1.0)

## Dead simple and super-efficient MIDI recommendation implementation

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/RecoMIDI
!pip install tqdm
!pip install pickle5
!pip install matplotlib
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install pretty_midi

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import tqdm as tqdm
import pickle5 as pickle

print('Loading TMIDIX module...')
os.chdir('/content/RecoMIDI')

import TMIDIX

import matplotlib.pyplot as plt
from midi2audio import FluidSynth
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from IPython.display import display, Javascript, HTML, Audio

from google.colab import output, drive

# (MIDI Dataset)

In [ ]:
#@title Download and untar full LAKH MIDI dataset


%cd /content/

print('=' * 70)
print('Downloading pre-trained dataset-model...Please wait...')
print('=' * 70)

!wget http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
print('=' * 70)

!tar -xvf lmd_full.tar.gz

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/

# (LAKH Pitches Signatures Pack)

In [ ]:
#@title Load LAKH Pitches Signatures Pack
LAKH = pickle.load(open('/content/RecoMIDI/LAKH_Signatures.pickle', 'rb'))

# (SOURCE MIDI)

In [ ]:
#@title Load your source MIDI
full_path_to_source_MIDI = "/content/lmd_full/8/80007bba6849fc49efd743f9b8dd19e6.mid" #@param {type:"string"}
f = full_path_to_source_MIDI

print('Loading MIDI file...')
score = TMIDIX.midi2score(open(f, 'rb').read())

events_matrix = []

itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
        if event[0] == 'note':
            events_matrix.append(event)
    itrack += 1
if len(events_matrix) > 0:
    instruments_pitches = [y[4] for y in events_matrix if y[3] != 9]
    instruments_pitches_count = len(instruments_pitches)
    instruments_pitches_sum = sum(instruments_pitches)
    
    drums_pitches = [y[4] for y in events_matrix if y[3] == 9]
    drums_pitches_count = len(drums_pitches)
    drums_pitches_sum = sum(drums_pitches)
    
    inputs = [f, instruments_pitches_count, instruments_pitches_sum, drums_pitches_count, drums_pitches_sum]
print('=' * 70)
print(inputs)


print('=' * 70)
print('Plotting the composition. Please wait...')

fname = f.split('.')[0]

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', fmin=1, hop_length=160, sr=16000, cmap=plt.cm.hot)
plt.title(fname)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# (FIND MATCHES)

In [ ]:
#@title Run to find matches in LAKH dataset

#@markdown PROTip: Use matches Rating to determine closeness to the source MIDI

similarity_rating = 0 #@param {type:"slider", min:0, max:0.00001, step:0.000001}
#@markdown similarity_rating == 0 will include exact matches

dissimilarity_rating = 0.0003 #@param {type:"slider", min:0.00001, max:0.001, step:0.00001}

print('RecoMIDI MIDI Signatures Matching Algorithm')
print('=' * 100)

found = []
fcount = 0

print('Source MIDI Signature', inputs[1:])
print('=' * 100)

print('Found Matches:')
print('=' * 100)

for L in LAKH:
  signature = abs(((inputs[2]+inputs[4]) / (inputs[1]+inputs[3])) - ((L[3]+L[5]) / (L[2]+L[4])))
  if fcount > 15:
    break
    
  if signature >= similarity_rating and signature <= dissimilarity_rating:
    print(fcount, L[1:], 'Rating:', signature)
    print('=' * 100)
    found.append(L)
    fcount += 1

In [ ]:
#@title Plot and listen to the desired match
which_match_to_plot = 0 #@param {type:"slider", min:0, max:15, step:1}

print('=' * 70)
print('Plotting the composition. Please wait...')

fo = found[which_match_to_plot][0].split('notebooks/')[1]

foo = '/content/' + fo
fname = foo.split('.')[0]

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', fmin=1, hop_length=160, sr=16000, cmap=plt.cm.hot)
plt.title(fname)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# Congrats! You did it! :)